In [1]:
import pandas as pd
import numpy as np
import random
import re
import time

In [2]:
"""
TODO:
download next dataframe > 24hrs after original (possibly use time.sleep())
produce timestamps of download times
stocks largest % change overall
stocks largest % change by exchange
industries largest % change
sectors largest % change

"""

'\nTODO:\ndownload next dataframe > 24hrs after original (possibly use time.sleep())\nproduce timestamps of download times\nstocks largest % change overall\nstocks largest % change by exchange\nindustries largest % change\nsectors largest % change\n\n'

In [3]:
def convert_to_snake_case(name):
    s1 = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', name)
    return re.sub('([a-z0-9])([A-Z])', r'\1_\2', s1).lower()

In [4]:
def create_stocks_df():
    combined = pd.DataFrame()
    exchange_names_strings_list = ["nasdaq","nyse","amex"]
    formatted_columns = []
    index_list = np.array([])
    
    for name in exchange_names_strings_list:
        exchange_tickers_url = "http://www.nasdaq.com/screening/companies-by-name.aspx?letter=0&exchange=" + \
        name + "&render=download"
        current = pd.read_csv(exchange_tickers_url)
        time.sleep(5) #take 5 seconds to ensure the API will not ban IP address.
        current.loc[:,"exchange"] = name.upper()
        if combined.empty:
            combined = combined.append(current)
        else:
            combined = pd.concat([combined,current])
    
    combined.dropna(axis=1,how="all",inplace=True)
    combined.drop(["Summary Quote","MarketCap","IPOyear"],axis=1,inplace=True)
    
    for col in combined.columns:
        formatted_columns.append(convert_to_snake_case(col))
    combined.columns = formatted_columns
    combined.rename({"last_sale":"price"},axis=1,inplace=True)
    combined.reset_index(inplace=True,drop=True)
    
    duplicates_bool = combined.loc[:,"symbol"].duplicated(keep=False)
    duplicates = combined.loc[duplicates_bool,:]
    
    nyse_duplicates_bool = duplicates.loc[:,"exchange"] == "NYSE"
    nasdaq_duplicates_bool = duplicates.loc[:,"exchange"] == "NASDAQ"
    amex_duplicates_bool = duplicates.loc[:,"exchange"] == "AMEX"

    nyse_duplicates_symbols = duplicates.loc[nyse_duplicates_bool,"symbol"]
    nasdaq_duplicates_symbols = duplicates.loc[nasdaq_duplicates_bool,"symbol"]
    amex_duplicates_symbols = duplicates.loc[amex_duplicates_bool,"symbol"]

    amex_in_nasdaq_bool = amex_duplicates_symbols.isin(nasdaq_duplicates_symbols)
    amex_in_nasdaq_true_indexes = (amex_in_nasdaq_bool == True).index.to_list()
    index_list = np.append(index_list,amex_in_nasdaq_true_indexes)

    nasdaq_in_nyse_bool = nasdaq_duplicates_symbols.isin(nyse_duplicates_symbols)
    nasdaq_in_nyse_true_indexes = (nasdaq_in_nyse_bool == True).index.to_list()
    index_list = np.append(index_list,nasdaq_in_nyse_true_indexes)

    index_list = index_list.astype(int)
    combined.drop(index_list,axis=0,inplace=True)
    
    return combined

In [5]:
stocks_original = create_stocks_df()
original_timestamp = pd.Timestamp.now
stocks_original.head()

,symbol,name,price,sector,industry,exchange
0,YI,"111, Inc.",2.79,Health Care,Medical/Nursing Services,NASDAQ
1,PIH,"1347 Property Insurance Holdings, Inc.",4.96,Finance,Property-Casualty Insurers,NASDAQ
2,PIHPP,"1347 Property Insurance Holdings, Inc.",26.61,Finance,Property-Casualty Insurers,NASDAQ
3,TURN,180 Degree Capital Corp.,2.05,Finance,Finance/Investors Services,NASDAQ
4,FLWS,"1-800 FLOWERS.COM, Inc.",18.26,Consumer Services,Other Specialty Stores,NASDAQ


In [6]:
stocks_original.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6934 entries, 0 to 6948
Data columns (total 6 columns):
symbol      6934 non-null object
name        6934 non-null object
price       6814 non-null float64
sector      5348 non-null object
industry    5348 non-null object
exchange    6934 non-null object
dtypes: float64(1), object(5)
memory usage: 379.2+ KB


In [21]:
stocks_later = stocks_original.copy()
later_timestamp = pd.Timestamp.now
stocks_later.drop(["name","sector","industry","exchange"],axis=1,inplace=True)
stocks_later.rename({"price":"new_price"},axis=1,inplace=True)

In [22]:
#Remove cell when in production, 'random' is only for testing purposes
stocks_later.loc[:,"new_price"] = round(stocks_later.loc[:,"new_price"] * (1.0 + random.uniform(-1.0,1.0)),4)
stocks_later.head()

,symbol,new_price
0,YI,3.97
1,PIH,7.06
2,PIHPP,37.86
3,TURN,2.92
4,FLWS,25.98


In [23]:
stocks_dif = stocks_original.merge(stocks_later,how="inner",on="symbol")
stocks_dif.loc[:,"$_price_change"] = stocks_dif.loc[:,"new_price"] - stocks_dif.loc[:,"price"]
stocks_dif.loc[:,"%_price_change"] = round((stocks_dif.loc[:,"$_price_change"] / stocks_dif.loc[:,"price"]),4)
stocks_dif = stocks_dif.sort_values(by="%_price_change",ascending=False)
stocks_dif.head()

,symbol,name,price,sector,industry,exchange,new_price,$_price_change,%_price_change
239,APDNW,Applied DNA Sciences Inc,0.0037,Consumer Services,Other Consumer Services,NASDAQ,0.01,0.0063,1.703
159,AMRWW,"Alta Mesa Resources, Inc.",0.0041,Energy,Oil & Gas Production,NASDAQ,0.01,0.0059,1.439
2891,SNOAW,"Sonoma Pharmaceuticals, Inc.",0.0053,Health Care,Major Pharmaceuticals,NASDAQ,0.01,0.0047,0.887
2783,SHIPW,Seanergy Maritime Holdings Corp,0.0130,Transportation,Marine Transportation,NASDAQ,0.02,0.0070,0.538
1077,EYEGW,"Eyegate Pharmaceuticals, Inc.",0.0398,Health Care,Major Pharmaceuticals,NASDAQ,0.06,0.0202,0.508


,price,new_price,$_price_change,%_price_change
exchange,,,,
AMEX,23.788927,36.645932,12.857005,0.540671
NASDAQ,30.048880,46.288810,16.239930,0.540401
NYSE,42.955012,66.170109,23.215097,0.540401
